In [14]:
import snap
import time, pandas as pd, pickle, json, networkx as nx, numpy as np
from networkx.readwrite import json_graph

In [16]:
cons = pd.read_csv("../REST/static/random_connections.csv", index_col="Unnamed: 0")
cons.sample(5)

,from_user_id,to_user_id,formation
13820,272006436,99515495,{'2018.05.07': True}
11090,123545757,437858563,{'2018.05.07': True}
9498,979272110,999498842,{'2018.05.05': True}
5394,548750474,985207736,{'2018.05.04': True}
10287,154837447,778977823,{'2018.05.06': True}


In [17]:
twitter_users = pd.read_csv("../REST/static/twitter_users.csv", index_col="id")
twitter_users.sample(5)

,match_name,screen_name,match_ratio,tw_id,name,lang,followers_count,friends_count,is_org,community
id,,,,,,,,,,
769479365,kariyer merkezi,ikmkariyer,95,7.694794e+17,İst. Kariyer Merkezi,tr,135.0,1065.0,False,33
757405478,mehmet arif ergun,ErgunMehmetA,91,7.574055e+17,Mehmet Ali Ergun,en,41.0,145.0,False,22
209749537,zafer demir,ZaferDemir_fr,100,2.097495e+08,Zafer Demir,fr,448.0,1812.0,False,36
152042796,ebubekir ceylan,ceylaneb,100,1.520428e+09,ebubekir ceylan,tr,2809.0,1259.0,False,3
915539516,merve akkus,merakks,91,9.155395e+08,merve akkuş,tr,60.0,247.0,False,27


In [21]:
nxg = nx.DiGraph()
truncate = lambda x: int(str(int(x))[:9])
for _, row in cons.iterrows():    
    from_ = truncate(row["from_user_id"])
    to = truncate(row["to_user_id"])
    if from_ in twitter_users.index and to in twitter_users.index:
        nxg.add_edge(from_, to)
nxg = nxg.to_directed()

In [2]:
# with open('../REST/static/networks/twitter_users_graph2.json', 'r') as f:
#     data = json.load(f)
#     nxg = json_graph.node_link_graph(data, directed=True)
# nxg = nxg.to_directed()

In [22]:
g = snap.PUNGraph.New()

In [23]:
for n in nxg.nodes():
    g.AddNode(n)

In [24]:
for f,t in nxg.edges():
    g.AddEdge(f, t)

In [25]:
CmtyV = snap.TCnComV()

## Calculating communities for nodes that had an edge at least once

In [ ]:
start = time.time()
modularity = snap.CommunityGirvanNewman(g, CmtyV)
print(time.time()-start)

In [ ]:
nodes_communities = {} # {node: [community]}
for i, Cmty in enumerate(CmtyV):
    print ("Community: ")
    for NI in Cmty:
        nodes_communities.setdefault(NI, [])
        nodes_communities[NI].append(i)
        print (NI)
print ("The modularity of the network is %f" % modularity)

In [ ]:
twitter_users["community"] = twitter_users.index.apply(lambda node: " ".join([str(com) for com in nodes_communities[node]]))
twitter_users.sample(5)

In [ ]:
twitter_users.to_csv("../REST/static/twitter_users.csv", index_label="id")

## Or, Augmenting the network directly

In [ ]:
for node in nxg.nodes():
    nxg.nodes[node]["community"] = " ".join([str(com) for com in nodes_communities[node]])

In [ ]:
def networkx_to_snappy(nxg, directed=False):
    if directed:
        g = snap.PNGraph.New()
    else:
        g = snap.PUNGraph.New()
        
    for n in nxg.nodes():
        g.AddNode(n)
    for f,t in nxg.edges():
        g.AddEdge(f, t)
        
    return g

In [ ]:
data = nx.node_link_data(nxg)
data["modularity"] = modularity
data["number_of_communities"] = len(CmtyV)

snappy_directed = networkx_to_snappy(nxg, True)
snappy_undirected = networkx_to_snappy(nxg, True)

diam_directed = snap.GetBfsFullDiam(snappy_directed, 100, True)
diam_undirected = snap.GetBfsFullDiam(snappy_undirected, 100, False)

data["diameter_directed"]=diam_directed
data["diameter_undirected"]=diam_undirected
with open('../visualization/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)
    
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)